In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#  LEGAL CASE SUMMARIZER — FULL WORKING VERSION (JSON INPUT ONLY)

from typing import List, Optional, Dict, Any, Tuple, Union
from pydantic import BaseModel, Field


#   SCHEMA DEFINITIONS

class Statute(BaseModel):
    code: str
    title: Optional[str] = None
    cited_text: Optional[str] = None
    confidence: Optional[float] = None


class SentenceItem(BaseModel):
    text: str
    span: Optional[List[int]] = None
    confidence: Optional[float] = None


class Relation(BaseModel):
    head: str
    relation: str
    tail: str
    evidence_spans: Optional[List[List[int]]] = None
    confidence: Optional[float] = None


class Parties(BaseModel):
    appellant: Optional[str] = None
    respondent: Optional[str] = None


class Metadata(BaseModel):
    label: Optional[str] = None
    court: Optional[str] = None
    bench: Optional[List[str]] = None
    decision_date: Optional[str] = None
    citation: Optional[str] = None
    case_no: Optional[str] = None
    parties: Optional[Parties] = None
    source_file: Optional[str] = None


class Arguments(BaseModel):
    Appellant: List[SentenceItem] = Field(default_factory=list)
    Respondent: List[SentenceItem] = Field(default_factory=list)
    Other: List[SentenceItem] = Field(default_factory=list)


class EnsembleCase(BaseModel):
    case_text: str
    statutes: List[Statute] = Field(default_factory=list)
    facts: List[SentenceItem] = Field(default_factory=list)
    issues: List[SentenceItem] = Field(default_factory=list)
    arguments: Arguments = Field(default_factory=Arguments)
    reasoning: List[SentenceItem] = Field(default_factory=list)
    order: List[SentenceItem] = Field(default_factory=list)
    relations: List[Relation] = Field(default_factory=list)
    metadata: Metadata = Field(default_factory=Metadata)


#   VALIDATION HELPERS

def _validate(model_cls, data):
    """Handles Pydantic v1/v2 safely."""
    if hasattr(model_cls, "model_validate"):
        return model_cls.model_validate(data)   # v2
    return model_cls.parse_obj(data)            # v1


def _fmt_statute(s: Statute) -> str:
    return f"{s.code} ({s.title})" if s.title else s.code


#   JSON → RUN_CASE ARGUMENTS

def to_run_case_args(ens: Union[EnsembleCase, Dict[str, Any]]):
    """Convert raw JSON → flattened arguments for run_case()."""

    if not isinstance(ens, EnsembleCase):
        ens = _validate(EnsembleCase, ens)

    case_text = ens.case_text.strip()

    statutes = [_fmt_statute(s) for s in ens.statutes]
    facts_sents = [x.text for x in ens.facts]
    issues_sents = [x.text for x in ens.issues]

    arguments_sents = []
    for party, items in [
        ("Appellant", ens.arguments.Appellant),
        ("Respondent", ens.arguments.Respondent),
        ("Other", ens.arguments.Other)
    ]:
        for it in items:
            arguments_sents.append(f"{party}: {it.text}")

    reasoning_sents = [x.text for x in ens.reasoning]
    order_sents = [x.text for x in ens.order]

    label = ens.metadata.label

    return (
        case_text, statutes,
        facts_sents, issues_sents,
        arguments_sents, reasoning_sents,
        order_sents, label
    )


#      SUMMARY GENERATOR

def _make_summary(case_text, statutes, facts_sents, issues_sents,
                  arguments_sents, reasoning_sents, order_sents):

    facts = " ".join(facts_sents)
    issue = issues_sents[0] if issues_sents else ""

    appellant = next((a.split(":",1)[1].strip()
                      for a in arguments_sents if a.startswith("Appellant:")), "")
    respondent = next((a.split(":",1)[1].strip()
                       for a in arguments_sents if a.startswith("Respondent:")), "")

    reasoning = " ".join(reasoning_sents)
    order = " ".join(order_sents)

    # ---------- TECHNICAL STRUCTURED SUMMARY ----------
    technical_summary = {
        "Court": "Supreme Court of India",
        "Statutes": statutes,
        "Facts": facts,
        "Issue": issue,
        "Arguments": {
            "Appellant": appellant,
            "Respondent": respondent
        },
        "Reasoning": reasoning,
        "Order": order
    }

    # ---------- LAYMAN SUMMARY (PARAGRAPH) ----------
    lay = []

    if facts:
        lay.append(f"The case started when {facts.lower()}.")
    if issue:
        lay.append(f"The main question before the court was whether {issue.lower()}.")
    if appellant:
        lay.append(f"The accused argued that {appellant.lower()}.")
    if respondent:
        lay.append(f"The complainant responded that {respondent.lower()}.")
    if reasoning:
        lay.append(f"The court reviewed the evidence and concluded that {reasoning.lower()}.")
    if order:
        lay.append(f"Finally, the court ruled that {order.lower()}.")

    layman_summary = " ".join(lay)

    # ---------- OPTIONAL ONE-LINER ----------
    legal_reasoning_one_liner = (
        "Under Section 420 IPC, the law requires fraudulent intent at the beginning of the transaction."
        if any("420" in s for s in statutes)
        else ""
    )

    return {
        "technical_summary": technical_summary,
        "layman_summary": layman_summary,
        "legal_reasoning_one_liner": legal_reasoning_one_liner
    }


#        FINAL RUN CASE

def run_case(case_text, statutes, facts_sents, issues_sents,
             arguments_sents, reasoning_sents, order_sents, label=None):

    summary = _make_summary(
        case_text, statutes,
        facts_sents, issues_sents,
        arguments_sents, reasoning_sents,
        order_sents
    )

    return {
        "ok": True,
        "label": label,
        "summary": summary
    }


# ===============================
#   WRAPPER (JSON ENTRY POINT)
# ===============================

def run_case_with_ensemble(run_case_fn, ens_json):
    args = to_run_case_args(ens_json)
    return run_case_fn(*args)


In [ ]:
def _make_summary(case_text, statutes, facts_sents, issues_sents, arguments_sents, reasoning_sents, order_sents):

    # ---------- Helpers ----------
    def sent(s):
        s = s.strip()
        return s if s.endswith(".") else s + "."

    def clean(s):
        return sent(s.replace("  ", " ").strip())

    def strip_whether(s):
        s = s.strip()
        if s.lower().startswith("whether "):
            return s[8:].rstrip(". ")
        return s.rstrip(". ")

    def lc_first(s):
        return s[0].lower() + s[1:] if s else s

    # ---------- Extract ----------
    facts = [clean(f) for f in facts_sents]
    issue_raw = issues_sents[0] if issues_sents else ""
    issue_core = strip_whether(issue_raw)
    issue_clean = clean(issue_raw)

    appellant_arg = next((a.split(":",1)[1].strip() for a in arguments_sents if a.startswith("Appellant:")), "")
    respondent_arg = next((a.split(":",1)[1].strip() for a in arguments_sents if a.startswith("Respondent:")), "")

    appellant_arg = clean(appellant_arg) if appellant_arg else ""
    respondent_arg = clean(respondent_arg) if respondent_arg else ""

    reasoning = [clean(r) for r in reasoning_sents]
    reasoning_text = " ".join(reasoning)
    order_text = clean(" ".join(order_sents)) if order_sents else ""

    #      TECHNICAL SUMMARY (JUDICIAL, FORMAL, LEGAL)
    tech = []

    if facts:
        tech.append(
            f"The present appeal before this Court arises from allegations wherein {lc_first(facts[0])}"
        )
        if len(facts) > 1:
            tech.append(" ".join(facts[1:]))

    if issue_core:
        tech.append(
            f"The central question requiring determination is whether {lc_first(issue_core)}."
        )

    if appellant_arg:
        tech.append(
            f"Learned counsel for the appellant submitted that {lc_first(appellant_arg)}"
        )
    if respondent_arg:
        tech.append(
            f"Conversely, learned counsel for the respondent contended that {lc_first(respondent_arg)}"
        )

    if reasoning_text:
        tech.append(
            f"Upon an evaluation of the evidentiary record and the statutory ingredients, the Court was of the view that {lc_first(reasoning_text)}"
        )

    if order_text:
        tech.append(
            f"In view of the foregoing analysis, the Court held that {lc_first(order_text)}"
        )

    technical_paragraph = " ".join(tech)

    #      LAYMAN SUMMARY (VERY SIMPLE, STORY FORMAT)
    lay = []

    # Story-like framing, different style
    if facts:
        lay.append(
            f"The problem started when {lc_first(facts[0]).replace(';', ',')}"
        )
        if len(facts) > 1:
            lay.append(" ".join(facts[1:]))

    if issue_core:
        lay.append(
            f"The court basically had to figure out whether {lc_first(issue_core)}."
        )

    if appellant_arg:
        lay.append(
            f"The accused person argued that {lc_first(appellant_arg)}"
        )

    if respondent_arg:
        lay.append(
            f"On the other hand, the complainant said that {lc_first(respondent_arg)}"
        )

    if reasoning_text:
        lay.append(
            f"After looking at everything, the judges felt that {lc_first(reasoning_text)}"
        )

    if order_text:
        lay.append(
            f"In simple terms, the court finally decided that {lc_first(order_text)}"
        )

    layman_summary = " ".join(lay)

    # Legal One-Liner
    legal_one_liner = (
        "Under Section 420 IPC, the prosecution must prove that the accused acted with dishonest intent at the very outset of the transaction."
        if any("420" in s for s in statutes)
        else ""
    )

    # Structured metadata for technical view
    technical_struct = {
        "Court": "Supreme Court of India",
        "Statutes": statutes,
        "Facts": " ".join(facts),
        "Issue": issue_clean,
        "Arguments": {"Appellant": appellant_arg, "Respondent": respondent_arg},
        "Reasoning": reasoning_text,
        "Order": order_text
    }

    return {
        "technical_summary": technical_struct,
        "technical_paragraph": technical_paragraph,
        "layman_summary": layman_summary,
        "legal_reasoning_one_liner": legal_one_liner,
    }


In [ ]:
# Cell 3: Test input JSON

test_input = {
    "case_text": "This case concerns allegations of cheating relating to a property transaction.",

    "statutes": [
        {"code": "Section 420 IPC", "title": "Cheating"},
        {"code": "Section 415 IPC", "title": "Definition of Cheating"}
    ],

    "facts": [
        {"text": "The complainant paid ₹12,00,000 after the accused claimed he had rights over the property."},
        {"text": "No registered sale deed was executed despite repeated follow-ups."}
    ],

    "issues": [
        {"text": "Whether the accused had dishonest intention at the beginning of the transaction."}
    ],

    "arguments": {
        "Appellant": [
            {"text": "The dispute is purely civil in nature; there was no intention to deceive."}
        ],
        "Respondent": [
            {"text": "The accused knew from the beginning that he had no title to the property."}
        ],
        "Other": []
    },

    "reasoning": [
        {"text": "Dishonest intent at the inception is the core requirement under Section 420 IPC."},
        {"text": "Evidence showed the accused had no legal right over the property."}
    ],

    "order": [
        {"text": "The appeal is dismissed. Conviction under Section 420 IPC is upheld."}
    ],

    "metadata": {"label": "Test Case – IPC 420"}
}


In [ ]:
# Cell 4: Run summarizer with clean paragraph formatting

output = run_case_with_ensemble(run_case, test_input)

# Full dictionary if needed
# output

# Pretty printing helper
def pretty_print(text):
    import textwrap
    wrapped = textwrap.fill(text, width=90)  # wrap at 90 chars per line
    print(wrapped)
    print("\n" + "-"*90 + "\n")

print("\n================ TECHNICAL PARAGRAPH ================\n")
pretty_print(output["summary"]["technical_paragraph"])

print("\n================ LAYMAN SUMMARY ================\n")
pretty_print(output["summary"]["layman_summary"])

print("\n================ LEGAL ONE-LINER ================\n")
pretty_print(output["summary"]["legal_reasoning_one_liner"])



================ TECHNICAL PARAGRAPH ================

The present appeal before this Court arises from allegations wherein the complainant paid
₹12,00,000 after the accused claimed he had rights over the property. No registered sale
deed was executed despite repeated follow-ups. The central question requiring
determination is whether the accused had dishonest intention at the beginning of the
transaction. Learned counsel for the appellant submitted that the dispute is purely civil
in nature; there was no intention to deceive. Conversely, learned counsel for the
respondent contended that the accused knew from the beginning that he had no title to the
property. Upon an evaluation of the evidentiary record and the statutory ingredients, the
Court was of the view that dishonest intent at the inception is the core requirement under
Section 420 IPC. Evidence showed the accused had no legal right over the property. The
appeal is dismissed. Conviction under Section 420 IPC is upheld.

------

Evaluation


In [ ]:
!pip install -q rouge-score

from rouge_score import rouge_scorer
import json
from statistics import mean

  Preparing metadata (setup.py) ... done


In [ ]:
import json
import os

EVAL_PATH = "/content/drive/MyDrive/legal/eval_cases.jsonl"  # change if you want

# Ensure the directory exists
output_dir = os.path.dirname(EVAL_PATH)
os.makedirs(output_dir, exist_ok=True)

eval_cases = [

    # ------- 1. Cheating in property transaction (similar theme, different wording) -------
    {
        "input": {
            "case_text": (
                "This appeal arises out of criminal proceedings alleging cheating in a "
                "property transaction. The complainant asserts that a substantial sum "
                "was paid to the accused on the representation that he had clear and "
                "marketable title. Subsequently, it emerged that the accused did not "
                "possess any legal right over the property in question."
            ),
            "statutes": [
                {"code": "Section 420 IPC", "title": "Cheating and dishonestly inducing delivery of property"},
                {"code": "Section 415 IPC", "title": "Definition of Cheating"}
            ],
            "facts": [
                {"text": "The complainant paid ₹15,00,000 to the accused on the assurance that the property was in his name."},
                {"text": "No registered sale deed was executed despite repeated requests and reminders."},
                {"text": "Later, revenue records revealed that the property never stood in the name of the accused."}
            ],
            "issues": [
                {"text": "Whether the ingredients of cheating under Section 420 IPC are satisfied."},
                {"text": "Whether the dispute is merely civil in nature or involves criminal liability."}
            ],
            "arguments": {
                "Appellant": [
                    {"text": "The dispute arises out of a failed sale transaction and at best amounts to breach of contract."},
                    {"text": "There was no dishonest intention at the inception of the transaction."}
                ],
                "Respondent": [
                    {"text": "The accused knew fully well that he had no title, yet induced payment by misrepresentation."},
                    {"text": "The conduct of the accused shows a fraudulent design from the very beginning."}
                ],
                "Other": []
            },
            "reasoning": [
                {"text": "Dishonest intention at the inception is the core requirement under Section 420 IPC."},
                {"text": "The accused never held title at any stage, and still represented that he was the owner."},
                {"text": "Failure to execute a sale deed combined with false assurances points to fraudulent intent rather than a mere civil dispute."}
            ],
            "order": [
                {"text": "The conviction under Section 420 IPC is affirmed and the appeal stands dismissed."}
            ],
            "metadata": {"label": "IPC 420 – Property Cheating"}
        },
        "reference": {
            "technical_paragraph": (
                "The appeal concerns allegations of cheating arising out of a property transaction in which the "
                "complainant paid ₹15,00,000 to the accused on the representation that he had clear title. No "
                "registered sale deed was ultimately executed, and subsequent verification of the revenue "
                "records revealed that the property never stood in the accused’s name. The primary issue before "
                "the Court was whether the dispute was purely civil or whether the statutory ingredients of "
                "Section 420 IPC were satisfied. While the appellant contended that the case reflected at best a "
                "breach of contractual obligations without any dishonest intent at inception, the respondent "
                "argued that the accused was fully aware of his lack of title and nevertheless induced payment "
                "through false representations. Upon examining the documentary material and the testimony on "
                "record, the Court concluded that the accused never possessed any legal right in the property, "
                "and the misrepresentation was made from the very beginning. Consequently, the Court held that "
                "dishonest intent at the inception stood established, upheld the conviction under Section 420 "
                "IPC and dismissed the appeal."
            ),
            "layman_summary": (
                "In this case, the complainant paid ₹15,00,000 to the accused because he was told that the "
                "property belonged to the accused. Later it turned out that the property was never in the "
                "accused’s name. The main question was whether this was just a simple property dispute or a "
                "criminal case of cheating. The accused said it was only a civil matter and that he never meant "
                "to cheat anyone. The complainant said the accused knew from day one that he had no right over "
                "the property but still took the money. After checking the documents and evidence, the Court "
                "agreed with the complainant. It held that the accused had dishonest intention from the "
                "beginning, treated the case as cheating under Section 420 IPC, upheld the conviction and "
                "dismissed the appeal."
            )
        }
    },

    # ------- 2. Dowry death 304B / 498A -------
    {
        "input": {
            "case_text": (
                "The prosecution alleges that the deceased was subjected to persistent cruelty and harassment in "
                "connection with unlawful demands of dowry, culminating in her unnatural death within seven "
                "years of marriage. The defence asserts that the death was accidental and unconnected with any "
                "dowry demand."
            ),
            "statutes": [
                {"code": "Section 304B IPC", "title": "Dowry Death"},
                {"code": "Section 498A IPC", "title": "Cruelty by husband or relatives of husband"},
                {"code": "Section 113B Evidence Act", "title": "Presumption as to dowry death"}
            ],
            "facts": [
                {"text": "The marriage took place three years prior to the incident."},
                {"text": "The deceased sustained burn injuries in her matrimonial home and succumbed thereafter."},
                {"text": "Witnesses stated that demands for a car and cash were repeatedly made by the husband and in-laws."}
            ],
            "issues": [
                {"text": "Whether the death of the deceased qualifies as a 'dowry death' under Section 304B IPC."},
                {"text": "Whether the statutory presumption under Section 113B of the Evidence Act stands attracted."}
            ],
            "arguments": {
                "Appellant": [
                    {"text": "The incident was an accidental kitchen fire and not connected to any dowry demand."},
                    {"text": "Witnesses are interested relatives whose testimony should not be relied upon."}
                ],
                "Respondent": [
                    {"text": "There was consistent evidence of dowry-related harassment soon before the death."},
                    {"text": "The death occurred within seven years of marriage, therefore the presumption under Section 113B applies."}
                ],
                "Other": []
            },
            "reasoning": [
                {"text": "For Section 304B IPC, the prosecution must show dowry-related cruelty 'soon before' death."},
                {"text": "Evidence of repeated demands for a car and cash coupled with threats indicates cruelty."},
                {"text": "The death in the matrimonial home within seven years of marriage triggers the statutory presumption."}
            ],
            "order": [
                {"text": "The conviction under Sections 304B and 498A IPC is upheld and the appeal is dismissed."}
            ],
            "metadata": {"label": "IPC 304B – Dowry Death"}
        },
        "reference": {
            "technical_paragraph": (
                "In the present appeal, the Court was required to examine whether the death of the deceased fell "
                "within the ambit of 'dowry death' under Section 304B IPC. The marriage had subsisted for about "
                "three years when the deceased sustained burn injuries in her matrimonial home and later died. "
                "The prosecution led evidence that the husband and his relatives persistently demanded a car and "
                "cash, and that the deceased was subjected to harassment on that account. The defence attempted "
                "to portray the incident as an accidental kitchen fire and disputed the credibility of the related "
                "witnesses. Having regard to the testimonies as well as the surrounding circumstances, the Court "
                "found that there was consistent evidence of dowry-related cruelty soon before the death. Since "
                "the death occurred within seven years of marriage, the presumption under Section 113B of the "
                "Evidence Act was attracted and was not rebutted by the appellants. Accordingly, the Court "
                "affirmed the conviction under Sections 304B and 498A IPC and dismissed the appeal."
            ),
            "layman_summary": (
                "Here, a woman died of burn injuries in her in-laws’ house about three years after her marriage. "
                "Her family said that the husband and his relatives kept demanding a car and cash and used to "
                "harass her for dowry. The husband’s side claimed that it was just an accidental kitchen fire and "
                "that the witnesses were biased relatives. The Court looked at all the statements and facts and "
                "felt that the dowry demands and harassment were real and had happened shortly before her death. "
                "Because the death happened within seven years of marriage, the law allows the Court to presume "
                "that it is a ‘dowry death’ unless the husband’s side can prove otherwise. They failed to do so, "
                "so the Court treated it as a dowry death under Section 304B IPC, upheld the conviction and "
                "rejected the appeal."
            )
        }
    },

    # ------- 3. Rape / consent and age (376, POCSO-like facts) -------
    {
        "input": {
            "case_text": (
                "The case concerns allegations of sexual assault wherein the prosecutrix, a minor at the time of "
                "the incident, alleges that the accused had repeated sexual intercourse with her on the false "
                "promise of marriage. The defence disputes her age and claims that the relationship was consensual."
            ),
            "statutes": [
                {"code": "Section 376 IPC", "title": "Punishment for rape"},
                {"code": "Section 375 IPC", "title": "Definition of rape"}
            ],
            "facts": [
                {"text": "School records and birth certificate indicate that the prosecutrix was 16 years old at the time of the incident."},
                {"text": "She stated that the accused established physical relations on the assurance of marriage."},
                {"text": "When the demand for marriage was repeated, the accused refused and denied the relationship."}
            ],
            "issues": [
                {"text": "Whether the prosecutrix was a minor at the relevant time."},
                {"text": "Whether the alleged consent was vitiated by a false promise of marriage."}
            ],
            "arguments": {
                "Appellant": [
                    {"text": "The prosecutrix was a consenting adult and the relationship was consensual."},
                    {"text": "The promise of marriage, if any, did not amount to deception at the inception."}
                ],
                "Respondent": [
                    {"text": "Documentary evidence conclusively shows that the prosecutrix was a minor."},
                    {"text": "Consent of a minor is no consent in the eyes of law, and the false promise of marriage further vitiates it."}
                ],
                "Other": []
            },
            "reasoning": [
                {"text": "School records and official documents are reliable indicators of age."},
                {"text": "Once it is established that the prosecutrix was a minor, consent is legally irrelevant."},
                {"text": "Even otherwise, consent obtained on a false promise of marriage may not be free and voluntary."}
            ],
            "order": [
                {"text": "The Court upholds the conviction of the accused under Section 376 IPC."}
            ],
            "metadata": {"label": "IPC 376 – Minor / consent"}
        },
        "reference": {
            "technical_paragraph": (
                "The Court was called upon to determine, first, whether the prosecutrix was a minor at the relevant "
                "time and, second, whether any alleged consent was vitiated by a false promise of marriage. Relying "
                "upon school records and the birth certificate, the Court held that the age of the prosecutrix was "
                "approximately 16 years when the incident occurred. In such circumstances, her apparent consent to "
                "sexual intercourse was legally immaterial, as consent of a minor is no consent in law. The Court "
                "further noted that the accused had repeatedly engaged in sexual relations on the assurance that he "
                "would marry her, but subsequently refused to do so and denied the relationship. On these facts, the "
                "Court concluded that the defence plea of a consensual relationship could not be accepted and upheld "
                "the conviction of the accused under Section 376 IPC."
            ),
            "layman_summary": (
                "In this case, the girl said that the accused had sexual relations with her many times after telling "
                "her that he would marry her. Later, he refused to marry and even denied the relationship. School "
                "records and her birth certificate showed that she was about 16 years old when all this happened. "
                "The Court said that if a girl is a minor, her ‘consent’ does not count in law. It also found that the "
                "promise of marriage was used only to get her to agree. Because of this, the Court did not accept the "
                "argument that the relationship was truly consensual and confirmed the conviction for rape under "
                "Section 376 IPC."
            )
        }
    },

    # ------- 4. Criminal breach of trust (406) -------
    {
        "input": {
            "case_text": (
                "The prosecution case is that the accused, who was entrusted with certain gold ornaments for "
                "safekeeping, dishonestly misappropriated the same and failed to return them on demand. The defence "
                "contends that the ornaments were pledged to discharge family debts with the knowledge of the "
                "complainant."
            ),
            "statutes": [
                {"code": "Section 406 IPC", "title": "Punishment for criminal breach of trust"},
                {"code": "Section 405 IPC", "title": "Criminal breach of trust"}
            ],
            "facts": [
                {"text": "The complainant handed over gold ornaments weighing 200 grams to the accused for safekeeping."},
                {"text": "When the complainant demanded the ornaments back, the accused failed to return them."},
                {"text": "There is no written consent or document authorising the accused to pledge or sell the ornaments."}
            ],
            "issues": [
                {"text": "Whether the entrustment of property and subsequent misappropriation are proved."},
                {"text": "Whether the act amounts to criminal breach of trust under Section 406 IPC."}
            ],
            "arguments": {
                "Appellant": [
                    {"text": "The ornaments were pledged to clear family debts with the complainant’s implied consent."},
                    {"text": "At best, there is a civil dispute over return of property and not a criminal offence."}
                ],
                "Respondent": [
                    {"text": "The accused was given custody only for safekeeping, not for pledge or sale."},
                    {"text": "Failure to return the ornaments upon demand shows dishonest misappropriation."}
                ],
                "Other": []
            },
            "reasoning": [
                {"text": "Entrustment is established once custody for a specific purpose is admitted by the accused."},
                {"text": "There is no material to show that the complainant authorised the accused to deal with the ornaments."},
                {"text": "Non-return of entrusted property coupled with absence of explanation constitutes misappropriation."}
            ],
            "order": [
                {"text": "The Court finds the ingredients of criminal breach of trust satisfied and upholds the conviction under Section 406 IPC."}
            ],
            "metadata": {"label": "IPC 406 – Criminal breach of trust"}
        },
        "reference": {
            "technical_paragraph": (
                "In this matter, the Court was required to consider whether the offence of criminal breach of trust "
                "under Section 406 IPC was made out. It was not disputed that the complainant had entrusted gold "
                "ornaments weighing 200 grams to the accused for the limited purpose of safekeeping. On being "
                "called upon to return the ornaments, the accused failed to do so and claimed that they had been "
                "pledged to clear family debts. However, there was no document or credible evidence showing that "
                "the complainant had authorised such a pledge. Once entrustment for a specific purpose is "
                "established, unauthorised dealing with the property and failure to restore it upon demand "
                "constitute dishonest misappropriation. The Court therefore held that the essential ingredients of "
                "criminal breach of trust stood proved and affirmed the conviction under Section 406 IPC."
            ),
            "layman_summary": (
                "Here, the complainant gave gold ornaments to the accused only to keep them safely. Later, when the "
                "complainant asked for the ornaments back, the accused did not return them and said that he had "
                "pledged them to pay off family debts. There was no proof that the complainant had ever allowed "
                "him to do that. The Court said that once someone accepts property just for safekeeping, he cannot "
                "use it for his own purpose without permission. Not returning the ornaments and giving no proper "
                "explanation showed that the accused had misused the property. So, the Court treated it as criminal "
                "breach of trust and confirmed the conviction under Section 406 IPC."
            )
        }
    },

    # ------- 5. Causing death by negligence (304A – road accident) -------
    {
        "input": {
            "case_text": (
                "The case arises from a road traffic accident in which the accused, while driving a bus, allegedly "
                "hit a pedestrian who later succumbed to injuries. The prosecution asserts rash and negligent "
                "driving; the defence contends that the victim suddenly came onto the road, leaving no time to react."
            ),
            "statutes": [
                {"code": "Section 304A IPC", "title": "Causing death by negligence"},
                {"code": "Section 279 IPC", "title": "Rash driving or riding on a public way"}
            ],
            "facts": [
                {"text": "The accused was driving a passenger bus on a busy road."},
                {"text": "Witnesses stated that the bus was being driven at a high speed and did not slow down near a pedestrian crossing."},
                {"text": "The pedestrian suffered serious injuries and died in hospital."}
            ],
            "issues": [
                {"text": "Whether the accused was driving in a rash and negligent manner."},
                {"text": "Whether the conduct of the accused amounts to causing death by negligence under Section 304A IPC."}
            ],
            "arguments": {
                "Appellant": [
                    {"text": "The victim suddenly ran onto the road, and the accident was unavoidable."},
                    {"text": "There is no concrete evidence of over-speeding or violation of traffic rules."}
                ],
                "Respondent": [
                    {"text": "Eyewitnesses consistently stated that the bus was driven at an excessive speed."},
                    {"text": "The driver failed to slow down near a crowded crossing, which shows negligence."}
                ],
                "Other": []
            },
            "reasoning": [
                {"text": "Rashness and negligence must be judged in the context of traffic conditions and duty of care owed by the driver."},
                {"text": "Driving a heavy vehicle at high speed near a pedestrian crossing without caution shows disregard for safety."},
                {"text": "The chain of events indicates that the breach of duty by the accused led directly to the victim’s death."}
            ],
            "order": [
                {"text": "The Court holds the accused guilty under Sections 279 and 304A IPC and maintains the sentence imposed by the trial court."}
            ],
            "metadata": {"label": "IPC 304A – Road accident"}
        },
        "reference": {
            "technical_paragraph": (
                "The present appeal concerns a road accident resulting in the death of a pedestrian allegedly caused "
                "by the rash and negligent driving of a bus by the accused. Eyewitnesses deposed that the bus was "
                "being driven at a high speed on a busy stretch and that the driver did not reduce speed on "
                "approaching a pedestrian crossing. The defence suggestion that the victim suddenly came onto the "
                "road and that the accident was unavoidable was not borne out by the overall evidence. The Court "
                "observed that a driver of a heavy vehicle owes a heightened duty of care, particularly in crowded "
                "areas. Operating the bus at excessive speed without adequate caution amounted to rashness and "
                "negligence. The Court thus held that the ingredients of Sections 279 and 304A IPC stood satisfied "
                "and upheld the conviction and sentence."
            ),
            "layman_summary": (
                "In this case, a bus driver was accused of causing a person’s death by driving carelessly. Witnesses "
                "said the bus was going very fast on a busy road and the driver did not slow down even near a place "
                "where people cross the road. The driver tried to argue that the pedestrian suddenly came onto the "
                "road and that he could not avoid the accident. The Court did not agree. It noted that someone who "
                "drives a big vehicle like a bus must be extra careful in crowded areas. Driving fast without "
                "slowing down near a crossing showed carelessness. The Court therefore held the driver guilty for "
                "rash and negligent driving and confirmed his conviction under Sections 279 and 304A IPC."
            )
        }
    }

]

# ---- Write to JSONL ----
with open(EVAL_PATH, "w", encoding="utf-8") as f:
    for ex in eval_cases:
        json.dump(ex, f, ensure_ascii=False)
        f.write("\n")

print(f"Saved {len(eval_cases)} examples to {EVAL_PATH}")

Saved 5 examples to /content/drive/MyDrive/legal/eval_cases.jsonl


In [ ]:
import json
from rouge_score import rouge_scorer
from statistics import mean

# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeL'],
    use_stemmer=True
)

def rouge_f(reference: str, prediction: str):
    """
    Returns only F1 scores for ROUGE-1, ROUGE-2, ROUGE-L.
    """
    scores = scorer.score(reference, prediction)
    return {
        "rouge1": scores["rouge1"].fmeasure,
        "rouge2": scores["rouge2"].fmeasure,
        "rougeL": scores["rougeL"].fmeasure,
    }


def evaluate_summarizer(eval_path: str):
    """
    Evaluates your summarizer on a dataset of cases in JSONL format.
    Each line must contain:
      {
        "input": {... case input ...},
        "reference": {
            "technical_paragraph": "...",
            "layman_summary": "..."
        }
      }
    """

    tech_r1, tech_r2, tech_rL = [], [], []
    lay_r1, lay_r2, lay_rL = [], [], []

    n_tech, n_lay = 0, 0

    with open(eval_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue

            ex = json.loads(line)

            case_json = ex["input"]
            ref = ex["reference"]

            # Run the summarizer
            out = run_case_with_ensemble(run_case, case_json)
            summ = out["summary"]

            # Technical paragraph evaluation
            if "technical_paragraph" in ref and ref["technical_paragraph"].strip():
                pred_tech = summ["technical_paragraph"]
                ref_tech = ref["technical_paragraph"]
                s = rouge_f(ref_tech, pred_tech)
                tech_r1.append(s["rouge1"])
                tech_r2.append(s["rouge2"])
                tech_rL.append(s["rougeL"])
                n_tech += 1

            # Layman summary evaluation
            if "layman_summary" in ref and ref["layman_summary"].strip():
                pred_lay = summ["layman_summary"]
                ref_lay = ref["layman_summary"]
                s = rouge_f(ref_lay, pred_lay)
                lay_r1.append(s["rouge1"])
                lay_r2.append(s["rouge2"])
                lay_rL.append(s["rougeL"])
                n_lay += 1

    results = {}

    if n_tech > 0:
        results["technical_paragraph"] = {
            "n_examples": n_tech,
            "rouge1": mean(tech_r1),
            "rouge2": mean(tech_r2),
            "rougeL": mean(tech_rL),
        }

    if n_lay > 0:
        results["layman_summary"] = {
            "n_examples": n_lay,
            "rouge1": mean(lay_r1),
            "rouge2": mean(lay_r2),
            "rougeL": mean(lay_rL),
        }

    return results

In [ ]:
results = evaluate_summarizer(EVAL_PATH)
results

{'technical_paragraph': {'n_examples': 5,
  'rouge1': 0.6029802511818537,
  'rouge2': 0.3270594658902351,
  'rougeL': 0.37059576642968317},
 'layman_summary': {'n_examples': 5,
  'rouge1': 0.49688311539202423,
  'rouge2': 0.20882045346606262,
  'rougeL': 0.3162231637112105}}

In [ ]:
import json
from rouge_score import rouge_scorer

EVAL_PATH = "/content/drive/MyDrive/legal/eval_cases.jsonl"  # same as before

def eval_per_example(eval_path):
    results = []
    with open(eval_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if not line.strip():
                continue
            ex = json.loads(line)
            case_id = ex["input"]["metadata"].get("label", f"case_{i}")
            ref = ex["reference"]

            out = run_case_with_ensemble(run_case, ex["input"])
            summ = out["summary"]

            tech_pred = summ["technical_paragraph"]
            lay_pred  = summ["layman_summary"]

            tech_ref = ref["technical_paragraph"]
            lay_ref  = ref["layman_summary"]

            tech_scores = scorer.score(tech_ref, tech_pred)
            lay_scores  = scorer.score(lay_ref,  lay_pred)

            results.append({
                "case_id": case_id,
                "tech_rouge1": tech_scores["rouge1"].fmeasure,
                "tech_rouge2": tech_scores["rouge2"].fmeasure,
                "tech_rougeL": tech_scores["rougeL"].fmeasure,
                "lay_rouge1":  lay_scores["rouge1"].fmeasure,
                "lay_rouge2":  lay_scores["rouge2"].fmeasure,
                "lay_rougeL":  lay_scores["rougeL"].fmeasure,
                "technical_pred": tech_pred,
                "technical_ref": tech_ref,
                "layman_pred": lay_pred,
                "layman_ref": lay_ref,
            })
    return results

per_case = eval_per_example(EVAL_PATH)
per_case

[{'case_id': 'IPC 420 – Property Cheating',
  'tech_rouge1': 0.6770025839793282,
  'tech_rouge2': 0.35844155844155845,
  'tech_rougeL': 0.44961240310077516,
  'lay_rouge1': 0.5201238390092878,
  'lay_rouge2': 0.22429906542056074,
  'lay_rougeL': 0.3467492260061919,
  'technical_pred': 'The present appeal before this Court arises from allegations wherein the complainant paid ₹15,00,000 to the accused on the assurance that the property was in his name. No registered sale deed was executed despite repeated requests and reminders. Later, revenue records revealed that the property never stood in the name of the accused. The central question requiring determination is whether the ingredients of cheating under Section 420 IPC are satisfied. Learned counsel for the appellant submitted that the dispute arises out of a failed sale transaction and at best amounts to breach of contract. Conversely, learned counsel for the respondent contended that the accused knew fully well that he had no title, 

In [ ]:
# Sort by layman ROUGE-1 (worst first)
sorted_by_lay = sorted(per_case, key=lambda x: x["lay_rouge1"])

for ex in sorted_by_lay:
    print("==" * 30)
    print("Case:", ex["case_id"])
    print(f"Layman ROUGE-1: {ex['lay_rouge1']:.3f}, ROUGE-2: {ex['lay_rouge2']:.3f}, ROUGE-L: {ex['lay_rougeL']:.3f}")
    print("\n[REF LAYMAN]\n", ex["layman_ref"])
    print("\n[PRED LAYMAN]\n", ex["layman_pred"])
    print("\n")

Case: IPC 376 – Minor / consent
Layman ROUGE-1: 0.457, ROUGE-2: 0.180, ROUGE-L: 0.243

[REF LAYMAN]
 In this case, the girl said that the accused had sexual relations with her many times after telling her that he would marry her. Later, he refused to marry and even denied the relationship. School records and her birth certificate showed that she was about 16 years old when all this happened. The Court said that if a girl is a minor, her ‘consent’ does not count in law. It also found that the promise of marriage was used only to get her to agree. Because of this, the Court did not accept the argument that the relationship was truly consensual and confirmed the conviction for rape under Section 376 IPC.

[PRED LAYMAN]
 The problem started when school records and birth certificate indicate that the prosecutrix was 16 years old at the time of the incident. She stated that the accused established physical relations on the assurance of marriage. When the demand for marriage was repeated, the

In [ ]:
# Sort by technical ROUGE-1 (worst first)
sorted_by_tech = sorted(per_case, key=lambda x: x["tech_rouge1"])

for ex in sorted_by_tech:
    print("==" * 30)
    print("Case:", ex["case_id"])
    print(
        f"Technical ROUGE-1: {ex['tech_rouge1']:.3f}, "
        f"ROUGE-2: {ex['tech_rouge2']:.3f}, "
        f"ROUGE-L: {ex['tech_rougeL']:.3f}"
    )

    print("\n[REF TECHNICAL]\n")
    print(ex["technical_ref"])

    print("\n[PRED TECHNICAL]\n")
    print(ex["technical_pred"])
    print("\n")

Case: IPC 406 – Criminal breach of trust
Technical ROUGE-1: 0.542, ROUGE-2: 0.287, ROUGE-L: 0.357

[REF TECHNICAL]

In this matter, the Court was required to consider whether the offence of criminal breach of trust under Section 406 IPC was made out. It was not disputed that the complainant had entrusted gold ornaments weighing 200 grams to the accused for the limited purpose of safekeeping. On being called upon to return the ornaments, the accused failed to do so and claimed that they had been pledged to clear family debts. However, there was no document or credible evidence showing that the complainant had authorised such a pledge. Once entrustment for a specific purpose is established, unauthorised dealing with the property and failure to restore it upon demand constitute dishonest misappropriation. The Court therefore held that the essential ingredients of criminal breach of trust stood proved and affirmed the conviction under Section 406 IPC.

[PRED TECHNICAL]

The present appeal 

All the technical predictions are very close in structure to the references:

They hit the same building blocks:

brief facts

core issue

appellant vs respondent arguments

court’s reasoning

final holding

The differences are mostly phrasing and intro framing.